# Store chunks into Vector Database using Azure Cognitive Search (ACS)

In [22]:
import os
import re
import pandas as pd
import json  
import openai  
from dotenv import dotenv_values
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from ast import literal_eval
import requests

In [7]:
# Choose name of index to create or update
index_name = "msft-transcripts"

# Load environment variables and keys 

In [12]:
# Specify the name of the .env file name 
env_name = "../../.env" # change to your own .env file name
config = dotenv_values(env_name)

if config['KEYS_FROM'] == "KEYVAULT":
    print('keyvault was selected.')
    keyVaultName = config["KEY_VAULT_NAME"]
    KVUri = f"https://{keyVaultName}.vault.azure.net"

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KVUri, credential=credential)

    # Azure OpenAI 
    openai.api_type = client.get_secret("OPENAI-API-TYPE").value
    openai.api_key = client.get_secret("OPENAI-API-KEY").value
    openai.api_base = client.get_secret("OPENAI-API-BASE").value
    openai.api_version = client.get_secret("OPENAI-API-VERSION").value
    deployment_embedding = client.get_secret("OPENAI-DEPLOYMENT-EMBEDDING").value

    ## Cog Search
    cogsearch_key = client.get_secret("COGSEARCH-API-KEY").value
    service_endpoint = client.get_secret("COGSEARCH-ADDRESS").value
else:
    # Azure OpenAI 
    openai.api_type = config["OPENAI_API_TYPE"] #"azure"
    openai.api_key = config['OPENAI_API_KEY']
    openai.api_base = config['OPENAI_API_BASE']
    openai.api_version = config['OPENAI_API_VERSION']
    deployment_embedding = config["OPENAI_DEPLOYMENT_EMBEDDING"]
    
    ## Cog Search
    cogsearch_key = config["COGSEARCH_API_KEY"]
    service_endpoint = config["COGSEARCH_ADDRESS"]


keyvault was selected.


In [13]:
def createEmbeddings(text, endpoint, api_key, api_version, embedding_model_deployment):
    request_url = f"{endpoint}/openai/deployments/{embedding_model_deployment}/embeddings?api-version={api_version}"
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    request_payload = {
        'input': text
    }
    embedding_response = requests.post(request_url, json=request_payload, headers=headers, timeout=None)
    if embedding_response.status_code == 200:
        data_values = embedding_response.json()["data"]
        embeddings_vectors = [data_value["embedding"] for data_value in data_values]
        return embeddings_vectors
    else:
        raise Exception(f"failed to get embedding: {embedding_response.json()}")


#### Store the embeddings in Azure Cognitive Search Vector Store

[AzureCogSearch](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search) provides a simple interface to create a vector database, store and retrieve data using vector search. You can read more about [here](https://github.com/Azure/cognitive-search-vector-pr/tree/main) more about Vector Search.

There are two steps to store data in AzureCogSearch vector database:
- First, we create the index (or schema) of the vector database
- Second, we add the chunked documents and their embeddings to the vector datastore

In [14]:
df_chunks_embedding = pd.read_csv('AnalyzedPDF/ChunksEmbedding.csv')

In [15]:
df_chunks_embedding.head(3)
#columns should look like the following with order preserved
#Id, Chunk, PageNumber, LineNumber, DocId, Embedding

,Id,Ticker,Year,Quarter,Chunk,PageNumber,LineNumber,Embedding
0,1,MSFT,23,1,Microsoft FY23 First Quarter Earnings Conferen...,1,1,"[-0.022691458, -0.02892966, -0.01939041, -0.02..."
1,2,MSFT,23,1,"On the Microsoft Investor Relations website, y...",1,9,"[-0.022940217, -0.0083436845, -0.008599305, -0..."
2,3,MSFT,23,1,GAAP. They are included as additional clarifyi...,1,17,"[-0.01130778, -0.0038822712, 0.003553209, -0.0..."


In [16]:

# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=AzureKeyCredential(cogsearch_key)
)
fields = [
    SimpleField(name="Id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="Ticker", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Year", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Quarter", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Chunk", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="PageNumber", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="LineNumber", type=SearchFieldDataType.String, filterable=True),
    
    SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="Ticker"),
        prioritized_content_fields=[SemanticField(field_name="Chunk")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 msft-transcripts created


In [17]:

## Upload data to Index
def batch_append_payload(df, search_client):
    """append payload for batch insertion (note: max 1000 rows per insertion) of embeddings to Cognitive Search"""
    value_list = []
    for index, row in df.iterrows():
        value_list.append(
            {
                "Id": str(index),
                "Ticker": row["Ticker"],
                "Year": str(row["Year"]),
                "Quarter": str(row["Quarter"]),
                "Chunk": row["Chunk"],
                "PageNumber": str(row["PageNumber"]),
                "LineNumber": str(row["LineNumber"]),
                "Embedding": literal_eval(row['Embedding']),
            }
        )
        
#         print(len(value_list))
        
        if len(value_list) >= 1000:
            result = search_client.upload_documents(value_list)
            print(f"Uploaded {len(value_list)} payload")
            value_list = []
    result = search_client.upload_documents(value_list)
    print(f"Uploaded {len(value_list)} payload")
    
            
            
#     print('payload of size {}'.format(len(value_list)))

    return value_list


search_client = SearchClient(
    endpoint=service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(cogsearch_key)
)
payload = batch_append_payload(df_chunks_embedding, search_client)
 
# print(f"Uploaded {len(payload)} payload") 


Uploaded 442 payload


# Search Types 1: Pure Vector Search

In [37]:
search_client = SearchClient(
	service_endpoint,
	index_name,
	credential=AzureKeyCredential(cogsearch_key)
)

In [38]:
# Pure Vector Search
query = "Microsoft earnings call for year 2023 for Quarter 2"  

query_embedding = createEmbeddings(
    query,
    openai.api_base,
    openai.api_key,
    openai.api_version,
    deployment_embedding
)[0]

vector = Vector(value=query_embedding, k=3, fields="Embedding")
  
results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    #select=["Ticker", "Quarter", "Year"],
    #top=3,
)

# results
  
for result in results: 
    print(result['Ticker'])
    print(result['Quarter'])
    print(result['Year'])
    print(result['Chunk'])
    break

MSFT
2
23
Microsoft FY23 Second Quarter Earnings Conference Call Brett Iversen, Satya Nadella, Amy Hood Tuesday, January 24, 2023 BRETT IVERSEN: Good afternoon and thank you for joining us today. On the call with me are Satya Nadella, chairman and chief executive officer, Amy Hood, chief financial officer, Alice Jolla, chief accounting officer, and Keith Dolliver, deputy general counsel. On the Microsoft Investor Relations website, you can find our earnings press release and financial summary slide deck, which is intended to 


# Search Types 2: Pure Filter

In [39]:
results = search_client.search(  
    search_text=None,  
    filter="(Ticker eq 'MSFT') and (Year eq '23') and (Quarter eq '1') ",
)  

for result in results:
    print(f"Ticker: {result['Ticker']}")
    print(f"Quarter: {result['Quarter']}") 
    print(f"Year: {result['Year']}") 
    print(result['Chunk'])
    print()
    break


Ticker: MSFT
Quarter: 1
Year: 23
Microsoft FY23 First Quarter Earnings Conference Call Brett Iversen, Satya Nadella, Amy Hood Tuesday, October 25, 2022 BRETT IVERSEN: Good afternoon and thank you for joining us today. On the call with me are Satya Nadella, chairman and chief executive officer, Amy Hood, chief financial officer, Alice Jolla, chief accounting officer, and Keith Dolliver, deputy general counsel. On the Microsoft Investor Relations website, you can find our earnings press release and financial summary slide deck, which is intended to 



# Search Types 3: Vector Search with filters

In [40]:
# Pure Vector Search with Filter
query = "What are the KPIs?"  
  
query_embedding = createEmbeddings(
    query,
    openai.api_base,
    openai.api_key,
    openai.api_version,
    deployment_embedding
)[0]

vector = Vector(value=query_embedding, k=3, fields="Embedding")  

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    filter="(Ticker eq 'MSFT') and (Year eq '23') and (Quarter eq '1') ",
    #select=["Ticker", "Quarter", "Year"],
    top=3,
)  
  
for result in results:
    print(f"Ticker: {result['Ticker']}")
    print(f"Quarter: {result['Quarter']}") 
    print(f"Year: {result['Year']}") 
    print(result['Chunk'])
    print()

    break

Ticker: MSFT
Quarter: 1
Year: 23
you're still seeing digitization. This is still the tailwind that helps customers solve problems. This is still the way to build growth and leverage in your business. And yet, you still want to optimize your workloads. You still want to run them the most efficiently so that you can then make room for new workload growth. We saw that across all segments. If there was one segment where I may have seen it a bit more, I would say, in the small or mid-sized segment of the market, that tends to be more through partner. We rely on partners to help customers do those same optimizations and prepare workloads. But it is that one point I know that people are focused on. 



# Search Types 4: Hybrid Search with filters

In [41]:
# Pure Vector Search with Filter
query = "What are the KPIs?"  
  
query_embedding = createEmbeddings(
    query,
    openai.api_base,
    openai.api_key,
    openai.api_version,
    deployment_embedding
)[0]

vector = Vector(value=query_embedding, k=3, fields="Embedding")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    filter="(Ticker eq 'MSFT') and (Year eq '23') and (Quarter eq '1') ",
#     select=["Ticker", "Quarter", "Year"],
    top=3,
)  
  
for result in results:
    print(f"Ticker: {result['Ticker']}")
    print(f"Quarter: {result['Quarter']}") 
    print(f"Year: {result['Year']}") 
    print(result['Chunk'])
    print()
    break

Ticker: MSFT
Quarter: 1
Year: 23
where we expected. And what we did see through the quarter is a real focus both by customers, but also by our sales and customer success teams on going proactively to customers and making sure we are helping them optimize their workloads. And as we went through the quarter and as, of course, the macroeconomic environment got more complicated, we continued to focus on that, because ultimately, those optimizations bring value even as budgets are still growing and budgeted spend is still growing. And so, it's this nuance of you're still seeing digitization. This is still the tailwind that helps customers solve problems. This is still the way to build growth and leverage in your 

